# Rozpoznawanie owoców i warzyw

Jako temat naszego projektu wybraliśmy rozpoznawanie różnych gatunków owoców i warzyw ze zdjęcia. 

## Dane techniczne

Komputer z system **Windows 11**

Procesor: **Intel(R) Core i5-6400 CPU Nvidia GForce GTX 1060** 

Język: **Python** 

Środowisko: **Jupyter notebook** 

Biblioteki: **PyTorch**

## Baza obrazów
Baza obrazów została pobrana ze strony [kaggle](https://www.kaggle.com/moltean/fruits). Obrazy w niej zawarte przedstawiają kilkadziesiąt różnych gatunków warzyw i owoców. Obrazy mają rozmiary 100 na 100 pikseli, i są robione na białym tle.

# Budowanie modelu

Sieć składa się z dwóch warstw konwolucyjnych, które bardzo ułatwiają analize obrazów, a nastepnie dwóch wartw "zwykłych", gdzie każdy neuron jest połączony z każdym nastepnym neuronem. Jako funckję aktywacji korzystaliśmy z wbudowanej funkcji ReLu.

In [2]:
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten
from torch.utils.data.dataloader import DataLoader

In [3]:
class LeNet(Module):
    def __init__(self, numChannels, classes):
        krnl_s = 5
        ch = 20
        
        super(LeNet, self).__init__()
        
        self.conv1 = Conv2d(in_channels=numChannels, out_channels=ch,
            kernel_size=(krnl_s, krnl_s))
        self.relu1 = ReLU()
        self.maxpool1 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        self.conv2 = Conv2d(in_channels=ch, out_channels=50,
            kernel_size=(krnl_s, krnl_s))
        self.relu2 = ReLU()
        self.maxpool2 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        self.fc1 = Linear(in_features=105800, out_features=500)
        self.relu3 = ReLU()
        
        self.fc2 = Linear(in_features=500, out_features=classes)
        self.logSoftmax = LogSoftmax(dim=1)

    def forward(self, x):
        
        x = self.conv1(x)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        
        x = flatten(x, 1)
        x = self.fc1(x)
        x = self.relu3(x)
        
        x = self.fc2(x)
        output = self.logSoftmax(x)
        
        return output
  

In [4]:
import matplotlib
matplotlib.use("Agg")
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torch.optim import Adam
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import torch

### Ustawienia
Obrazki były dzielone na grupy po 64. Sieć była uczona przez 2 epochy, ponieważ później następowało przetrenowanie. 75% danych było wykorzystywane do uczenia a 25% do walidacji.

In [8]:
INIT_LR = 1e-3
BATCH_SIZE = 64
EPOCHS = 2

TRAIN_SPLIT = 0.75
VAL_SPLIT = 1 - TRAIN_SPLIT

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Ładowanie danych


In [10]:
trainData  = ImageFolder('proj1/archive/fruits-360_dataset/fruits-360/Training', transform=ToTensor())

numTrainSamples = int(len(trainData) * TRAIN_SPLIT)
numValSamples = int(len(trainData) * VAL_SPLIT)
(trainData, valData) = random_split(trainData,
	[numTrainSamples, numValSamples],
	generator=torch.Generator().manual_seed(42))

trainDataLoader = DataLoader(trainData, shuffle=True, batch_size=BATCH_SIZE)
valDataLoader = DataLoader(valData, batch_size=BATCH_SIZE)

In [12]:
testData  = ImageFolder('proj1/archive/fruits-360_dataset/fruits-360/Test', transform=ToTensor())
testDataLoader = DataLoader(testData, batch_size=BATCH_SIZE)

trainSteps = len(trainDataLoader.dataset) // BATCH_SIZE
valSteps = len(valDataLoader.dataset) // BATCH_SIZE

In [13]:
model = LeNet(numChannels=3, classes=len(trainData.dataset.classes)).to(device)


opt = Adam(model.parameters(), lr=INIT_LR)
lossFn = nn.NLLLoss()

H = {
	"train_loss": [],
	"train_acc": [],
	"val_loss": [],
	"val_acc": []
}

# Uczenie modelu

In [ ]:

for e in range(0, EPOCHS):
	model.train()
    
	totalTrainLoss = 0
	totalValLoss = 0
    
	trainCorrect = 0
	valCorrect = 0
    
	for (x, y) in trainDataLoader:
        
		(x, y) = (x.to(device), y.to(device))
        
		pred = model(x)
		loss = lossFn(pred, y)
        
		opt.zero_grad()
		loss.backward()
		opt.step()
        
		totalTrainLoss += loss
		trainCorrect += (pred.argmax(1) == y).type(
			torch.float).sum().item()
        
        
	with torch.no_grad():
        
		model.eval()
        
		for (x, y) in valDataLoader:
            
			(x, y) = (x.to(device), y.to(device))
            
			pred = model(x)
			totalValLoss += lossFn(pred, y)
            
			valCorrect += (pred.argmax(1) == y).type(
				torch.float).sum().item()
            
            
	avgTrainLoss = totalTrainLoss / trainSteps
	avgValLoss = totalValLoss / valSteps
    
	trainCorrect = trainCorrect / len(trainDataLoader.dataset)
	valCorrect = valCorrect / len(valDataLoader.dataset)
    
	H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
	H["train_acc"].append(trainCorrect)
	H["val_loss"].append(avgValLoss.cpu().detach().numpy())
	H["val_acc"].append(valCorrect)
    
	print("[INFO] EPOCH: {}/{}".format(e + 1, EPOCHS))
	print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(
		avgTrainLoss, trainCorrect))
	print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(
		avgValLoss, valCorrect))

# Wykres poprawności modelu

In [10]:
plt.style.use("ggplot")
plt.figure()
plt.plot(H["train_loss"], label="train_loss")
plt.plot(H["val_loss"], label="val_loss")
plt.plot(H["train_acc"], label="train_acc")
plt.plot(H["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")

plt.savefig(f"plot_save_{EPOCHS}epoch.pdf")
torch.save(model, f"model_{EPOCHS}epoch")

Poniższy wykres prezentuje dokładność sieci neuronowej dla danych testowych i dla danych walidacyjnych (przy których wagi w sieci się nie zmieniały). Oś pozioma wskazuje epochy (od 0), a pionowa dokładność w skali od 0 do 1.

![wykres](plot_save_2epoch-1.png)

## Testowanie pojedynczych obrazków:
W bazie danych był też dostępny zbiór dodatkowych obrazów do testowania. Najpierw sprawdziliśmy jak sieć radzi sobie z rozpoznawaniem pojedynczych obrazków.

![cherry](test_dataset_cherry.jpg)
![cherry](test_dataset_banana.jpg)
![cherry](test_dataset_apple.jpg)
![cherry](test_dataset_avocado.jpg)
![cherry](test_dataset_peach.jpg)

Dodatkowo zrobiliśmy zdjęcie papryki, poddaliśmy lekkiej obróbce (zmniejszyliśmy rozmiar, ustawiliśmy tło na białe, i lekko zmieniliśmy kolorystykę ze względu na inne warunki oświetleniowe) i ją też przetestowaliśmy.

![cherry](papryka100x100_postprocess.jpg)

In [17]:
from PIL import Image
from torchvision import transforms

def label_single_image(model_path, image_path, my_model = None):
	if my_model == None:
		my_model = torch.load(model_path, map_location=torch.device('cpu')).to(device)
	with torch.no_grad():
        
		my_model.eval()

		img = Image.open(image_path)
		
		convert_tensor = transforms.ToTensor()
		img_tensor = convert_tensor(img)
		img_tensor = img_tensor.unsqueeze(0)

		img_tensor = img_tensor.to(device)
		pred = my_model(img_tensor)

		label_id = pred.max(1).indices
		for l_id in label_id:
			return testData.classes[l_id]


In [26]:

print(label_single_image("model_2epoch", "test_dataset_cherry.jpg"))
print(label_single_image("model_2epoch", "papryka100x100_postprocess.jpg"))
print(label_single_image("model_2epoch", "test_dataset_banana.jpg"))
print(label_single_image("model_2epoch", "test_dataset_apple.jpg"))
print(label_single_image("model_2epoch", "test_dataset_avocado.jpg"))
print(label_single_image("model_2epoch", "test_dataset_peach.jpg"))

Cherry 1
Pepper Red
Banana
Apple Braeburn
Avocado
Peach


Jak widać dla tych obrazków sieć wskazywała poprawne etykiety.

# Testowanie dużej ilości danych
Przetestowaliśmy też większą ilość obrazków ze zbioru testowego. Jak widać poniżej otrzymaliśmy dokładność 97.84871616932685%. Dodatkowo warto zauważyć że duża ilość błędów (poniżej kodu wypisane są niedopasowania) wynikają z pogrupowania danych w bazie np. istnieją dwie etykiety "Apple Red 1" i "Apple Red 2", które sieć traktuje jako dwa osobne owoce.

In [23]:
import os

def test_all_files(dir):
    my_model = torch.load("model_2epoch", map_location=torch.device('cpu')).to(device)

    all_count = 0
    correct_count = 0
    for file in os.scandir(dir):
        if file.is_dir():
            label = file.name
            img_count = 0
            for img in os.scandir(file):
                all_count += 1
                res = label_single_image("", img.path, my_model)
                if label != res:
                    print(img.path, "|", res, "!=", label)
                else:
                    correct_count +=1

                if img_count < 10:
                    img_count += 1
                else:
                    break
    return correct_count / all_count

print(test_all_files("proj1/archive/fruits-360_dataset/fruits-360/Test") * 100, end="%")

proj1/archive/fruits-360_dataset/fruits-360/Test\Apple Red 1\321_100.jpg | Apple Red 2 != Apple Red 1
proj1/archive/fruits-360_dataset/fruits-360/Test\Apple Red 1\322_100.jpg | Apple Red 2 != Apple Red 1
proj1/archive/fruits-360_dataset/fruits-360/Test\Apple Red 1\323_100.jpg | Apple Red 2 != Apple Red 1
proj1/archive/fruits-360_dataset/fruits-360/Test\Cantaloupe 2\161_100.jpg | Apple Golden 1 != Cantaloupe 2
proj1/archive/fruits-360_dataset/fruits-360/Test\Cantaloupe 2\163_100.jpg | Apple Golden 1 != Cantaloupe 2
proj1/archive/fruits-360_dataset/fruits-360/Test\Eggplant\100_100.jpg | Plum 3 != Eggplant
proj1/archive/fruits-360_dataset/fruits-360/Test\Eggplant\101_100.jpg | Plum 3 != Eggplant
proj1/archive/fruits-360_dataset/fruits-360/Test\Eggplant\102_100.jpg | Plum 3 != Eggplant
proj1/archive/fruits-360_dataset/fruits-360/Test\Eggplant\104_100.jpg | Plum 3 != Eggplant
proj1/archive/fruits-360_dataset/fruits-360/Test\Eggplant\105_100.jpg | Plum 3 != Eggplant
proj1/archive/fruits-360_

## Wnioski
Otrzymana sieć bardzo dobrze radziła sobie na obrazkach z bazy danych, jednak były one poddane dużej obróbce także sieć prawdopodobnie nie radziła by sobie dla zdjęć zrobionych w innych warunkach.